In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tentativa-anterior/resposta.csv
/kaggle/input/santander-customer-satisfaction/sample_submission.csv
/kaggle/input/santander-customer-satisfaction/train.csv
/kaggle/input/santander-customer-satisfaction/test.csv


In [2]:
dataset = pd.read_csv("/kaggle/input/santander-customer-satisfaction/train.csv")

### 0. Preparativos
Antes de tentar uma outra abordagem, é interessante avaliar o resultado passado.

In [140]:
tentativa = pd.read_csv("/kaggle/input/tentativa-anterior/resposta.csv")

In [4]:
tentativa.TARGET.value_counts()

TARGET
0    55931
1    19887
Name: count, dtype: int64

De maneira interessante, a resposta enviada teve uma alta incidência de clientes insatisfeitos (TARGET = 1) se comparado com o conjunto de dados entregue: cerca de 26.2%, contra os quase 4% do conjunto de dados. A técnica de oversampling pode ter gerado uma tendência no modelo final, contribuindo para esse resultado. Dessa vez, vamos tentar uma abordagem diferente.

### 1. Criando as Amostras
Dessa vez, faremos uso do algoritmo SelectKBest para escolher as colunas adequadas, e treinaremos um conjunto com o Oversampling, e outro com o Undersampling, para avaliar a diferença de resultados na previsão.

In [7]:
dataset.TARGET.value_counts()

TARGET
0    73012
1     3008
Name: count, dtype: int64

Primeiro, vamos eliminar as já conhecidas colunas com valores constantes.

In [3]:
colunas_constantes = [coluna for coluna in dataset.columns if dataset[coluna].nunique() == 1]
dataset = dataset.drop(columns = colunas_constantes)

In [5]:
dataset.shape

(76020, 337)

Vamos também separar as colunas mais relevantes do conjunto de dados, deixando apenas as outras características.

In [8]:
dataset.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [11]:
ID = dataset.ID
dataset = dataset.drop(columns = "ID")

In [12]:
TARGET = dataset.TARGET
dataset = dataset.drop(columns = "TARGET")

In [13]:
dataset.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016


Também será necessário escalar o conjunto de dados.

In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

dataset_scaled = scaler.fit_transform(dataset.copy())

dataset_scaled = pd.DataFrame(dataset_scaled, columns = dataset.columns)
dataset_scaled.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.0,0.01513,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


#### 1.1 Usando SelectKBest
Na última tentativa, conjuntos de maior tamanho (Entre 60 e 40 colunas) apresentaram um melhor resultado na avaliação de modelos. Assim, vamos tentar, dessa vez, usar o algoritmo para extrair um conjunto maior de colunas e avaliar o seu desempenho.

In [32]:
from sklearn.feature_selection import SelectKBest, chi2

In [44]:
seletor = SelectKBest(chi2, k = 25)
resultado = seletor.fit_transform(dataset_scaled, TARGET)
resultado = pd.DataFrame(resultado, columns=dataset.columns[seletor.get_support()])

SELECT_25_BEST = set(resultado.columns)

In [39]:
seletor = SelectKBest(chi2, k = 50)
resultado = seletor.fit_transform(dataset_scaled, TARGET)
resultado = pd.DataFrame(resultado, columns=dataset.columns[seletor.get_support()])

SELECT_50_BEST = set(resultado.columns)

In [41]:
seletor = SelectKBest(chi2, k = 75)
resultado = seletor.fit_transform(dataset_scaled, TARGET)
resultado = pd.DataFrame(resultado, columns=dataset.columns[seletor.get_support()])

SELECT_75_BEST = set(resultado.columns)

In [42]:
seletor = SelectKBest(chi2, k = 100)
resultado = seletor.fit_transform(dataset_scaled, TARGET)
resultado = pd.DataFrame(resultado, columns=dataset.columns[seletor.get_support()])

SELECT_100_BEST = set(resultado.columns)

In [43]:
seletor = SelectKBest(chi2, k = 150)
resultado = seletor.fit_transform(dataset_scaled, TARGET)
resultado = pd.DataFrame(resultado, columns=dataset.columns[seletor.get_support()])

SELECT_150_BEST = set(resultado.columns)

#### 1.2 Undersampling e Oversampling
Agora, vamos separar o conjunto de dados em duas categorias distintas.

In [46]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [47]:
SEED = 912039

In [48]:
smote = SMOTE(random_state = SEED)
x_smote, y_smote = smote.fit_resample(dataset_scaled, TARGET)

In [49]:
dataset_smote = pd.concat([x_smote, y_smote], axis = 1)

In [50]:
dataset_smote.TARGET.value_counts()

TARGET
0    73012
1    73012
Name: count, dtype: int64

In [51]:
dataset_smote.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545,0
1,0.999764,0.29,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003,0
2,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822,0
3,0.999764,0.32,0.0,0.01513,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671,0
4,0.999764,0.34,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091,0


In [52]:
sampler = RandomUnderSampler(random_state = SEED)
x_under, y_under = sampler.fit_resample(dataset_scaled, TARGET)

In [53]:
dataset_under = pd.concat([x_under, y_under], axis = 1)

In [54]:
dataset_under.TARGET.value_counts()

TARGET
0    3008
1    3008
Name: count, dtype: int64

In [55]:
dataset_under.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
4797,0.999764,0.18,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091,0
55973,0.999764,0.27,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005048,0
58919,0.999764,0.24,0.0,0.043997,0.086713,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001438,0
16143,0.999764,0.17,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091,0
49809,0.999764,0.20,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091,0


#### 1.3 Finalizando a Preparação
Agora, vamos separar os nossos alvos para teste.

In [56]:
y_oversampling = dataset_smote.TARGET
y_undersampling = dataset_under.TARGET

dataset_smote.drop("TARGET", axis = 1, inplace = True)
dataset_under.drop("TARGET", axis = 1, inplace = True)

In [62]:
oversampling_25 = pd.DataFrame({coluna: dataset_smote[coluna] for coluna in SELECT_25_BEST})
oversampling_50 = pd.DataFrame({coluna: dataset_smote[coluna] for coluna in SELECT_50_BEST})
oversampling_75 = pd.DataFrame({coluna: dataset_smote[coluna] for coluna in SELECT_75_BEST})
oversampling_100 = pd.DataFrame({coluna: dataset_smote[coluna] for coluna in SELECT_100_BEST})
oversampling_150 = pd.DataFrame({coluna: dataset_smote[coluna] for coluna in SELECT_150_BEST})

In [63]:
undersampling_25 = pd.DataFrame({coluna: dataset_under[coluna] for coluna in SELECT_25_BEST})
undersampling_50 = pd.DataFrame({coluna: dataset_under[coluna] for coluna in SELECT_50_BEST})
undersampling_75 = pd.DataFrame({coluna: dataset_under[coluna] for coluna in SELECT_75_BEST})
undersampling_100 = pd.DataFrame({coluna: dataset_under[coluna] for coluna in SELECT_100_BEST})
undersampling_150 = pd.DataFrame({coluna: dataset_under[coluna] for coluna in SELECT_150_BEST})

Agora que temos os alvos de treinamento definidos, vamos ao próprio treinamento.

### 2. Treinando Modelos

In [64]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

linear = SVC(random_state = SEED)
knn = KNeighborsClassifier()
lr = LogisticRegression(random_state = SEED)
dt = DecisionTreeClassifier(random_state = SEED)
rf = RandomForestClassifier(random_state = SEED)
ab = AdaBoostClassifier(random_state = SEED)
gb = GradientBoostingClassifier(random_state = SEED)
extra = ExtraTreesClassifier(random_state = SEED)
mlp = MLPClassifier(random_state = SEED)

A função de avaliação também será modificada para refletir o valor da curva ROC, a métrica da competição.

In [71]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, roc_auc_score

def avalia_modelo(modelo, x, y):
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1',
        'roc_auc': make_scorer(roc_auc_score)
    }
    
    resultados = cross_validate(modelo, x, y, scoring=scoring, cv=10, n_jobs=-1)
    
    print(f'Modelo: {modelo}')
    media_acuracia = sum(resultados['test_accuracy'])/10
    media_precisao = sum(resultados['test_precision'])/10
    media_recall = sum(resultados['test_recall'])/10
    media_f1 = sum(resultados['test_f1'])/10
    media_roc_auc = sum(resultados['test_roc_auc'])/10
    
    print(f'Acurácia --> {media_acuracia :.2f}')
    print(f'Precisão --> {media_precisao :.2f}')
    print(f'Recall --> {media_recall :.2f}')
    print(f'F1 --> {media_f1 :.2f}')
    print(f'ROC AUC --> {media_roc_auc :.2f}')

#### 4.1 KNN

In [72]:
avalia_modelo(knn, oversampling_25, y_oversampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.69
Precisão --> 0.95
Recall --> 0.41
F1 --> 0.57
ROC AUC --> 0.69


In [74]:
avalia_modelo(knn, oversampling_50, y_oversampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.69
Precisão --> 0.95
Recall --> 0.41
F1 --> 0.57
ROC AUC --> 0.69


In [75]:
avalia_modelo(knn, oversampling_75, y_oversampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.70
Precisão --> 0.91
Recall --> 0.44
F1 --> 0.59
ROC AUC --> 0.70


In [76]:
avalia_modelo(knn, oversampling_100, y_oversampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.70
Precisão --> 0.89
Recall --> 0.46
F1 --> 0.61
ROC AUC --> 0.70


In [77]:
avalia_modelo(knn, oversampling_150, y_oversampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.71
Precisão --> 0.88
Recall --> 0.49
F1 --> 0.63
ROC AUC --> 0.71


In [78]:
avalia_modelo(knn, undersampling_25, y_undersampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.56
Precisão --> 0.67
Recall --> 0.25
F1 --> 0.36
ROC AUC --> 0.56


In [79]:
avalia_modelo(knn, undersampling_50, y_undersampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.56
Precisão --> 0.66
Recall --> 0.26
F1 --> 0.37
ROC AUC --> 0.56


In [80]:
avalia_modelo(knn, undersampling_75, y_undersampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.60
Precisão --> 0.67
Recall --> 0.39
F1 --> 0.49
ROC AUC --> 0.60


In [81]:
avalia_modelo(knn, undersampling_100, y_undersampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.60
Precisão --> 0.67
Recall --> 0.39
F1 --> 0.49
ROC AUC --> 0.60


In [82]:
avalia_modelo(knn, undersampling_150, y_undersampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.59
Precisão --> 0.65
Recall --> 0.40
F1 --> 0.49
ROC AUC --> 0.59


O aumento no número de amostras (colunas) impactou levemente as métricas positivamente. A abordagem de undersampling foi severamente inferior.

#### 4.2 Árvore de Decisão

In [83]:
avalia_modelo(dt, oversampling_25, y_oversampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.87
Precisão --> 0.89
Recall --> 0.85
F1 --> 0.86
ROC AUC --> 0.87


In [84]:
avalia_modelo(dt, oversampling_50, y_oversampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.87
Precisão --> 0.88
Recall --> 0.85
F1 --> 0.86
ROC AUC --> 0.87


In [85]:
avalia_modelo(dt, oversampling_75, y_oversampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.87
Precisão --> 0.87
Recall --> 0.86
F1 --> 0.86
ROC AUC --> 0.87


In [86]:
avalia_modelo(dt, oversampling_100, y_oversampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.87
Precisão --> 0.87
Recall --> 0.86
F1 --> 0.87
ROC AUC --> 0.87


In [87]:
avalia_modelo(dt, oversampling_150, y_oversampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.87
Precisão --> 0.87
Recall --> 0.88
F1 --> 0.87
ROC AUC --> 0.87


In [88]:
avalia_modelo(dt, undersampling_25, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.71
Recall --> 0.68
F1 --> 0.69
ROC AUC --> 0.70


In [89]:
avalia_modelo(dt, undersampling_50, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.71
Recall --> 0.68
F1 --> 0.69
ROC AUC --> 0.70


In [90]:
avalia_modelo(dt, undersampling_75, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.70
F1 --> 0.71
ROC AUC --> 0.71


In [91]:
avalia_modelo(dt, undersampling_100, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.70
F1 --> 0.71
ROC AUC --> 0.71


In [92]:
avalia_modelo(dt, undersampling_150, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.72
F1 --> 0.71
ROC AUC --> 0.71


O mesmo se repete, com a Árvore de Decisão sendo, novamente, um algoritmo de boa performance.

#### 4.3 Regressão Logística

In [98]:
avalia_modelo(lr, oversampling_25, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.68
Recall --> 0.74
F1 --> 0.71
ROC AUC --> 0.70


In [99]:
avalia_modelo(lr, oversampling_50, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.68
Recall --> 0.74
F1 --> 0.71
ROC AUC --> 0.70


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [100]:
avalia_modelo(lr, oversampling_75, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.69
Recall --> 0.75
F1 --> 0.72
ROC AUC --> 0.70


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [101]:
avalia_modelo(lr, oversampling_100, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.69
Recall --> 0.75
F1 --> 0.72
ROC AUC --> 0.71


In [102]:
avalia_modelo(lr, oversampling_150, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.70
Recall --> 0.75
F1 --> 0.73
ROC AUC --> 0.71


In [93]:
avalia_modelo(lr, undersampling_25, y_undersampling)

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.68
Recall --> 0.73
F1 --> 0.71
ROC AUC --> 0.70


In [94]:
avalia_modelo(lr, undersampling_50, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.68
Recall --> 0.73
F1 --> 0.71
ROC AUC --> 0.70


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [95]:
avalia_modelo(dt, undersampling_75, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.70
F1 --> 0.71
ROC AUC --> 0.71


In [96]:
avalia_modelo(dt, undersampling_100, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.70
F1 --> 0.71
ROC AUC --> 0.71


In [97]:
avalia_modelo(dt, undersampling_150, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.72
F1 --> 0.71
ROC AUC --> 0.71


A performance da Regressão Logística não parece ter sido muito afetada. Os algoritmos de árvore parecem ser os mais adequados para o problema até então.

#### 4.4 RandomForest

In [103]:
avalia_modelo(rf, oversampling_25, y_oversampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.86
Precisão --> 0.88
Recall --> 0.84
F1 --> 0.86
ROC AUC --> 0.86


In [104]:
avalia_modelo(rf, oversampling_50, y_oversampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.86
Precisão --> 0.88
Recall --> 0.84
F1 --> 0.86
ROC AUC --> 0.86


In [105]:
avalia_modelo(rf, oversampling_75, y_oversampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.87
Precisão --> 0.88
Recall --> 0.86
F1 --> 0.87
ROC AUC --> 0.87


In [106]:
avalia_modelo(rf, oversampling_100, y_oversampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.88
Precisão --> 0.89
Recall --> 0.87
F1 --> 0.88
ROC AUC --> 0.88


In [107]:
avalia_modelo(rf, oversampling_150, y_oversampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.88
Precisão --> 0.88
Recall --> 0.87
F1 --> 0.88
ROC AUC --> 0.88


In [108]:
avalia_modelo(rf, undersampling_25, y_undersampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.71
F1 --> 0.71
ROC AUC --> 0.71


In [109]:
avalia_modelo(rf, undersampling_50, y_undersampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.71
Recall --> 0.70
F1 --> 0.71
ROC AUC --> 0.71


In [110]:
avalia_modelo(rf, undersampling_75, y_undersampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.74
Precisão --> 0.74
Recall --> 0.73
F1 --> 0.73
ROC AUC --> 0.74


In [111]:
avalia_modelo(rf, undersampling_100, y_undersampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.73
Precisão --> 0.74
Recall --> 0.72
F1 --> 0.73
ROC AUC --> 0.73


In [112]:
avalia_modelo(rf, undersampling_150, y_undersampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.75
Precisão --> 0.76
Recall --> 0.72
F1 --> 0.74
ROC AUC --> 0.75


O uso de mais amostras (colunas) parece ser um benefício a mais para algoritmos ensemble. Undersampling continua apresentando um desempenho menor.

#### 4.5 MLP

In [113]:
avalia_modelo(mlp, undersampling_25, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Modelo: MLPClassifier(random_state=912039)
Acurácia --> 0.73
Precisão --> 0.73
Recall --> 0.73
F1 --> 0.73
ROC AUC --> 0.73


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [115]:
avalia_modelo(mlp, undersampling_50, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

Modelo: MLPClassifier(random_state=912039)
Acurácia --> 0.73
Precisão --> 0.73
Recall --> 0.72
F1 --> 0.72
ROC AUC --> 0.73


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [117]:
avalia_modelo(mlp, undersampling_75, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

Modelo: MLPClassifier(random_state=912039)
Acurácia --> 0.74
Precisão --> 0.74
Recall --> 0.74
F1 --> 0.74
ROC AUC --> 0.74


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [118]:
avalia_modelo(mlp, undersampling_100, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

Modelo: MLPClassifier(random_state=912039)
Acurácia --> 0.73
Precisão --> 0.75
Recall --> 0.70
F1 --> 0.73
ROC AUC --> 0.73


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [119]:
avalia_modelo(mlp, undersampling_150, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

Modelo: MLPClassifier(random_state=912039)
Acurácia --> 0.74
Precisão --> 0.74
Recall --> 0.73
F1 --> 0.73
ROC AUC --> 0.74


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Undersampling não gerou resultados em nenhum modelo. O uso de maiores conjuntos de colunas pode provar-se benéfico.

### 3. Gerando os Resultados

In [122]:
alvo = pd.read_csv("/kaggle/input/santander-customer-satisfaction/test.csv")

alvo_id = alvo.ID

O melhor resultado de Oversampling foi obtido com 100 colunas, e de Undersampling, com 150.

In [126]:
scaler = MinMaxScaler()

copia = alvo.copy()
copia = scaler.fit_transform(copia)

alvo = pd.DataFrame(copia, columns = alvo.columns)
alvo.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.000000,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.000020,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.000026,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.000033,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.000046,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [127]:
amostra_over = pd.DataFrame({coluna: alvo[coluna] for coluna in SELECT_100_BEST})
amostra_under = pd.DataFrame({coluna: alvo[coluna] for coluna in SELECT_150_BEST})

In [128]:
modelo_over = RandomForestClassifier(random_state = SEED)
modelo_under = RandomForestClassifier(random_state = SEED)

In [129]:
modelo_over.fit(oversampling_100, y_oversampling)

RandomForestClassifier(random_state=912039)

In [130]:
modelo_under.fit(undersampling_150, y_undersampling)

RandomForestClassifier(random_state=912039)

In [131]:
resultados_over = modelo_over.predict(amostra_over)

In [132]:
resposta_over = pd.concat([alvo_id, pd.Series(resultados_over, name = "TARGET")], axis = 1)

In [133]:
resposta_over

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,1


In [134]:
resultados_under = modelo_under.predict(amostra_under)

In [135]:
resposta_under = pd.concat([alvo_id, pd.Series(resultados_under, name = "TARGET")], axis = 1)

In [136]:
resposta_under

,ID,TARGET
0,2,0
1,5,0
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


#### 3.1 Comparando os resultados

In [137]:
resposta_over.TARGET.value_counts()

TARGET
0    50100
1    25718
Name: count, dtype: int64

In [138]:
resposta_under.TARGET.value_counts()

TARGET
0    58964
1    16854
Name: count, dtype: int64

In [141]:
tentativa.TARGET.value_counts()

TARGET
0    55931
1    19887
Name: count, dtype: int64

In [142]:
resposta_over.to_csv("resposta_over.csv", index = False)

In [143]:
resposta_under.to_csv("resposta_under.csv", index = False)